In [16]:
import os
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader  # ✅ fixed here
from torchvision import transforms
from tqdm import tqdm
import torch.nn as nn
import torchvision.models as models
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [17]:
import pickle
BASE_PATH = "/kaggle/input/cifar-10-and-cifar-10-c/cifar-10-python/cifar-10-batches-py"

train_data = []
train_label = []

def unpickle(file):
    with open(file ,'rb') as f:
        data_dict = pickle.load(f , encoding = 'bytes')
    return data_dict
        
for i in range(1,6):
    batch_path = os.path.join(BASE_PATH , f"data_batch_{i}" )
    batch = unpickle(batch_path)
    train_data.append(batch[b'data'])
    train_label.extend(batch[b'labels'])

X_train = np.vstack(train_data)      
y_train = np.array(train_label)     

In [18]:
test_data = []
test_label = []

test_path = os.path.join(BASE_PATH , 'test_batch')
test_batch = unpickle(test_path)
test_data.append(batch[b'data'])
test_label.extend(batch[b'labels'])

y_test = np.array(test_label)    

In [19]:
CIFAR_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR_STD  = (0.2470, 0.2435, 0.2616)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),         
    transforms.RandomHorizontalFlip(p=0.5),        
    transforms.ColorJitter(brightness=0.1, contrast=0.1,
                           saturation=0.1, hue=0.1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD)
])

In [20]:
def cifar_flat_to_pil(img_flat):
    r = img_flat[0:1024].reshape(32,32)
    g = img_flat[1024:2048].reshape(32,32)
    b = img_flat[2048:3072].reshape(32,32)
    img = np.stack([r,g,b], axis=2).astype('uint8')
    pil_img = Image.fromarray(img).convert("RGB")
    return pil_img

In [21]:
from PIL import Image
def cifar_dataset_to_pil(flat_images):
    pil_images = []
    for img_flat in flat_images:
        pil_img = cifar_flat_to_pil(img_flat)  # pass the array directly
        pil_images.append(pil_img)
    return pil_images

train_data_flat = np.vstack(train_data)
test_data_flat = np.vstack(test_data)   

train_images = cifar_dataset_to_pil(train_data_flat)
test_images = cifar_dataset_to_pil(test_data_flat)

In [22]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
import multiprocessing

class CIFAR10_dataset(Dataset):
    def __init__(self, images, labels=None, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        self.has_labels = labels is not None
        print(f"\nLoading CIFAR10 Dataset... Total Len: {len(images)} | Labeled: {self.has_labels}\n{'-'*50}")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        if self.transform:
            img = self.transform(img)
        if self.has_labels:
            label = torch.tensor(self.labels[idx], dtype=torch.long)
            return {"img": img, "label": label}
        else:
            return {"img": img}


train_images_split, val_images, y_train_split, y_val = train_test_split(
    train_images, y_train,
    test_size=0.2,         
    random_state=42,       
    stratify=y_train        
)

In [23]:
train_dataset = CIFAR10_dataset(train_images_split, y_train_split, transform=train_transform)
val_dataset = CIFAR10_dataset(val_images, y_val, transform=test_transform)
test_dataset = CIFAR10_dataset(test_images, labels=None, transform=test_transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=multiprocessing.cpu_count(),
    pin_memory=True,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=128,
    shuffle=False,
    num_workers=multiprocessing.cpu_count(),
    pin_memory=True,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=128,
    shuffle=False,
    num_workers=multiprocessing.cpu_count(),
    pin_memory=True,
    persistent_workers=True
)


Loading CIFAR10 Dataset... Total Len: 40000 | Labeled: True
--------------------------------------------------

Loading CIFAR10 Dataset... Total Len: 10000 | Labeled: True
--------------------------------------------------

Loading CIFAR10 Dataset... Total Len: 10000 | Labeled: False
--------------------------------------------------


In [24]:
def create_model(model_name, num_classes=10):
    if model_name == 'ResNet18':
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == 'EfficientNet-B0':
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif model_name == 'VGG16':
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(4096, num_classes)
    
    return model

In [25]:
model_configs = {
    'ResNet18': {'optimizer': 'SGD', 'lr': 0.01, 'momentum': 0.9},
    'EfficientNet-B0': {'optimizer': 'Adam', 'lr': 0.001},
    'VGG16': {'optimizer': 'Adam', 'lr': 0.001}}

In [28]:
import os

# Create results directory
os.makedirs('/kaggle/working/model_results', exist_ok=True)

# ==================== MODEL DEFINITIONS ====================
def create_model(model_name, num_classes=10):
    if model_name == 'ResNet18':
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    elif model_name == 'EfficientNet-B0':
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    elif model_name == 'VGG16':
        model = models.vgg16(pretrained=True)
        model.classifier[6] = nn.Linear(4096, num_classes)
    
    return model

# Models to train
model_configs = {
    'ResNet18': {'optimizer': 'SGD', 'lr': 0.01, 'momentum': 0.9},
    'EfficientNet-B0': {'optimizer': 'Adam', 'lr': 0.001},
    'VGG16': {'optimizer': 'Adam', 'lr': 0.0005}
}

# ==================== TRAINING FUNCTION ====================
def train_model(model, model_name, train_loader, val_loader, epochs=50):
    print(f"Training {model_name}")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    config = model_configs[model_name]
    if config['optimizer'] == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=config['lr'], 
                                  weight_decay=1e-4, momentum=config.get('momentum', 0.9))
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], 
                                   weight_decay=1e-4, betas=(0.9, 0.999))
    
    criterion = nn.CrossEntropyLoss()
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, min_lr=0.00001)
    
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': [],
        'val_precision': [], 'val_recall': [], 'val_f1': []
    }
    
    best_val_acc = 0
    best_model_path = f"/kaggle/working/model_results/best_{model_name.replace('-', '_')}.pth"
    
    for epoch in range(epochs):
        
        model.train()
        train_loss, train_correct = 0, 0
        all_train_preds, all_train_labels = [], []
        
        for batch in tqdm(train_loader, desc=f'{model_name} - Epoch {epoch+1}/{epochs} [Train]'):
            images, labels = batch["img"].to(device), batch["label"].to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * images.size(0)
            preds = outputs.argmax(1)
            train_correct += (preds == labels).sum().item()
            
            all_train_preds.extend(preds.cpu().numpy())
            all_train_labels.extend(labels.cpu().numpy())
        
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = 100. * train_correct / len(train_loader.dataset)
        
        # Validation
        model.eval()
        val_loss, val_correct = 0, 0
        all_val_preds, all_val_labels, all_val_probs = [], [], []
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f'{model_name} - Epoch {epoch+1}/{epochs} [Val]'):
                images, labels = batch["img"].to(device), batch["label"].to(device)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * images.size(0)
                preds = outputs.argmax(1)
                val_correct += (preds == labels).sum().item()
                
                probs = torch.softmax(outputs, dim=1)
                
                all_val_preds.extend(preds.cpu().numpy())
                all_val_labels.extend(labels.cpu().numpy())
                all_val_probs.extend(probs.cpu().numpy())
        
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = 100. * val_correct / len(val_loader.dataset)
        
        # Compute metrics
        from sklearn.metrics import precision_score, recall_score, f1_score
        
        val_precision = precision_score(all_val_labels, all_val_preds, average='macro', zero_division=0)
        val_recall = recall_score(all_val_labels, all_val_preds, average='macro', zero_division=0)
        val_f1 = f1_score(all_val_labels, all_val_preds, average='macro', zero_division=0)
        
        # Store history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['val_precision'].append(val_precision)
        history['val_recall'].append(val_recall)
        history['val_f1'].append(val_f1)
        
        print(f"\n{model_name} - Epoch {epoch+1} Results:")
        print(f"   Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"   Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        print(f"   Val Precision: {val_precision:.4f} | Recall: {val_recall:.4f} | F1: {val_f1:.4f}")
        print(f"   LR: {optimizer.param_groups[0]['lr']:.6f}")
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"NEW BEST MODEL! Val Acc: {val_acc:.2f}%")
            
            # Create model-specific directory
            model_dir = f"/kaggle/working/model_results/{model_name}"
            os.makedirs(model_dir, exist_ok=True)
            
            # Save classification report
            class_names = ['airplane', 'auto', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
            report = classification_report(all_val_labels, all_val_preds, target_names=class_names, output_dict=True)
            report_df = pd.DataFrame(report).transpose()
            report_df.to_csv(f'{model_dir}/{model_name}_classification_report.csv')
            
            # Save confusion matrix
            cm = confusion_matrix(all_val_labels, all_val_preds)
            np.savetxt(f'{model_dir}/{model_name}_confusion_matrix.csv', cm, delimiter=',', fmt='%d')
            
            # Save training plots for this model
            plt.figure(figsize=(15, 5))
            
            plt.subplot(1, 3, 1)
            plt.plot(history['train_loss'][:epoch+1], label='Train Loss', linewidth=2)
            plt.plot(history['val_loss'][:epoch+1], label='Val Loss', linewidth=2)
            plt.title(f'{model_name} - Loss')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.subplot(1, 3, 2)
            plt.plot(history['train_acc'][:epoch+1], label='Train Acc', linewidth=2)
            plt.plot(history['val_acc'][:epoch+1], label='Val Acc', linewidth=2)
            plt.title(f'{model_name} - Accuracy')
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy (%)')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.subplot(1, 3, 3)
            plt.plot(history['val_f1'][:epoch+1], label='Val F1', color='red', linewidth=2)
            plt.title(f'{model_name} - F1 Score')
            plt.xlabel('Epoch')
            plt.ylabel('F1 Score')
            plt.legend()
            plt.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig(f'{model_dir}/{model_name}_training_plots.png', dpi=120, bbox_inches='tight')
            plt.close()
        
        lr_scheduler.step(val_loss)
    
    print(f"\n{model_name} Training Complete! Best Val Acc: {best_val_acc:.2f}%")
    
    return history, best_val_acc

# ==================== TRAIN ALL MODELS ====================
all_results = {}

for model_name in model_configs.keys():
    model = create_model(model_name)
    history, best_acc = train_model(model, model_name, train_loader, val_loader, epochs=50)
    all_results[model_name] = {
        'history': history,
        'best_val_acc': best_acc
    }
    
    # Save final training history for each model
    model_dir = f"/kaggle/working/model_results/{model_name}"
    history_df = pd.DataFrame(history)
    history_df.to_csv(f'{model_dir}/{model_name}_training_history.csv', index=False)
    print(f"{model_name} training history saved to {model_dir}")

# ==================== FINAL SUMMARY ====================
print("TRAINING COMPLETE - SUMMARY")

for model_name, result in all_results.items():
    best_acc = result['best_val_acc']
    best_f1 = max(result['history']['val_f1'])
    print(f"{model_name}:")
    print(f"   Best Validation Accuracy: {best_acc:.2f}%")
    print(f"   Best F1 Score: {best_f1:.4f}")
    print(f"   Results saved to: /kaggle/working/model_results/{model_name}/")
    print()

print("All model results saved successfully!")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 178MB/s] 


Training ResNet18


ResNet18 - Epoch 1/50 [Val]: 100%|██████████| 79/79 [00:01<00:00, 61.72it/s]



ResNet18 - Epoch 1 Results:
   Train Loss: 1.3326 | Train Acc: 53.36%
   Val Loss: 1.0739 | Val Acc: 62.18%
   Val Precision: 0.6892 | Recall: 0.6218 | F1: 0.6177
   LR: 0.010000
NEW BEST MODEL! Val Acc: 62.18%


ResNet18 - Epoch 2/50 [Val]: 100%|██████████| 79/79 [00:00<00:00, 79.04it/s]


...


VGG16 - Epoch 48 Results:
   Train Loss: 0.1784 | Train Acc: 94.45%
   Val Loss: 0.3333 | Val Acc: 90.54%
   Val Precision: 0.9064 | Recall: 0.9054 | F1: 0.9057
   LR: 0.000050
NEW BEST MODEL! Val Acc: 90.54%


VGG16 - Epoch 49/50 [Val]: 100%|██████████| 79/79 [00:02<00:00, 30.07it/s]



VGG16 - Epoch 49 Results:
   Train Loss: 0.1476 | Train Acc: 95.26%
   Val Loss: 0.3362 | Val Acc: 90.80%
   Val Precision: 0.9086 | Recall: 0.9080 | F1: 0.9082
   LR: 0.000050
NEW BEST MODEL! Val Acc: 90.80%


VGG16 - Epoch 50/50 [Val]: 100%|██████████| 79/79 [00:02<00:00, 30.86it/s]


VGG16 - Epoch 50 Results:
   Train Loss: 0.1472 | Train Acc: 95.28%
   Val Loss: 0.3244 | Val Acc: 90.80%
   Val Precision: 0.9084 | Recall: 0.9080 | F1: 0.9081
   LR: 0.000050

VGG16 Training Complete! Best Val Acc: 90.80%
VGG16 training history saved to /kaggle/working/model_results/VGG16
TRAINING COMPLETE - SUMMARY
ResNet18:
   Best Validation Accuracy: 86.13%
   Best F1 Score: 0.8617
   Results saved to: /kaggle/working/model_results/ResNet18/

EfficientNet-B0:
   Best Validation Accuracy: 87.93%
   Best F1 Score: 0.8797
   Results saved to: /kaggle/working/model_results/EfficientNet-B0/

VGG16:
   Best Validation Accuracy: 90.80%
   Best F1 Score: 0.9082
   Results saved to: /kaggle/working/model_results/VGG16/

All model results saved successfully!


In [ ]:
# KAGGLE gave a freeze error so i am gonna run the rest of the code in other NOTEBOOK